# SPR 2026 - Sentence Transformers

**SBERT: embeddings densos de alta qualidade**

- ✅ paraphrase-multilingual-MiniLM-L12-v2
- ✅ Embeddings 384D pré-treinados
- ✅ Tempo esperado: ~5-10 min

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. Add Data → **Models** → `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`
3. **IMPORTANTE:** Execute "Run All" após commit

> O notebook auto-detecta o modelo em `/kaggle/input/`.
---

In [ ]:
# =============================================================================
# SPR 2026 - SBERT: SENTENCE TRANSFORMERS + LIGHTGBM
# =============================================================================
# - paraphrase-multilingual-MiniLM-L12-v2 (offline)
# - Embeddings 384D
# - LightGBM classifier
# =============================================================================

import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# =============================================================================
# AUTO-DETECTAR MODELO EM /kaggle/input (BUSCA RECURSIVA)
# =============================================================================
def find_sbert_model(base='/kaggle/input'):
    """
    Procura o modelo SBERT em /kaggle/input.
    Busca recursivamente por pastas contendo modules.json (SentenceTransformer).
    """
    if not os.path.exists(base):
        return None
    
    def is_sbert_folder(path):
        """Verifica se a pasta contém um modelo SentenceTransformer válido"""
        if not os.path.isdir(path):
            return False
        return os.path.exists(os.path.join(path, 'modules.json'))
    
    def search_recursive(path, depth=0, max_depth=6):
        """Busca recursiva até max_depth níveis"""
        if depth > max_depth or not os.path.isdir(path):
            return None
        
        if is_sbert_folder(path):
            return path
        
        try:
            for item in os.listdir(path):
                item_path = os.path.join(path, item)
                if os.path.isdir(item_path):
                    result = search_recursive(item_path, depth + 1, max_depth)
                    if result:
                        return result
        except PermissionError:
            pass
        
        return None
    
    return search_recursive(base)

MODEL_PATH = find_sbert_model()

np.random.seed(SEED)
print('[1/5] Bibliotecas carregadas!')
print('DATA_DIR ->', DATA_DIR)

# Debug: mostrar estrutura COMPLETA do /kaggle/input
def print_tree(path, prefix='', depth=0, max_depth=4):
    """Mostra árvore de diretórios com indicação de modules.json"""
    if depth > max_depth or not os.path.isdir(path):
        return
    
    try:
        items = sorted(os.listdir(path))
        for i, item in enumerate(items[:10]):  # Limita a 10 itens por pasta
            item_path = os.path.join(path, item)
            is_last = (i == len(items[:10]) - 1)
            connector = '└── ' if is_last else '├── '
            
            if os.path.isdir(item_path):
                has_modules = ' ✅ SBERT!' if os.path.exists(os.path.join(item_path, 'modules.json')) else ''
                print(f'{prefix}{connector}📂 {item}/{has_modules}')
                extension = '    ' if is_last else '│   '
                print_tree(item_path, prefix + extension, depth + 1, max_depth)
            else:
                if item in ['modules.json', 'config.json', 'model.safetensors']:
                    print(f'{prefix}{connector}📄 {item}')
        
        if len(items) > 10:
            print(f'{prefix}... (+{len(items)-10} mais arquivos)')
    except PermissionError:
        pass

print('\n📁 Estrutura de /kaggle/input:')
base = '/kaggle/input'
if os.path.exists(base):
    print_tree(base)

print()
if MODEL_PATH:
    print(f'✅ MODEL_PATH -> {MODEL_PATH}')
    print(f'   Arquivos: {os.listdir(MODEL_PATH)[:10]}')
else:
    print('❌ Modelo SBERT não encontrado!')
    print('\n🔧 COMO RESOLVER:')
    print('   1. Remove o input atual (se houver)')
    print('   2. Add Data → Models')
    print('   3. Procure: "paraphrase-multilingual-MiniLM-L12-v2"')
    print('   4. Escolha: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    print('   5. Save Version → Save & Run All')

# =============================================================================
# CARREGAR DADOS
# =============================================================================
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f'[2/5] Train: {train.shape} | Test: {test.shape}')

# =============================================================================
# CARREGAR SENTENCE TRANSFORMER
# =============================================================================
if MODEL_PATH is None:
    raise FileNotFoundError(
        "Modelo SBERT não encontrado em /kaggle/input.\n\n"
        "COMO RESOLVER:\n"
        "  1. Add Data → Models\n"
        "  2. Procure: paraphrase-multilingual-MiniLM-L12-v2\n"
        "  3. Selecione: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2\n"
        "  4. Save Version → Save & Run All"
    )

model = SentenceTransformer(MODEL_PATH)
print(f'[3/5] Modelo SentenceTransformer carregado de {MODEL_PATH}')

# =============================================================================
# GERAR EMBEDDINGS
# =============================================================================
print('Gerando embeddings do treino...')
X_train = model.encode(train['report'].tolist(), show_progress_bar=True, batch_size=32)
y_train = train['target'].values

print('Gerando embeddings do teste...')
X_test = model.encode(test['report'].tolist(), show_progress_bar=True, batch_size=32)
print(f'[4/5] Embeddings: X_train {X_train.shape} | X_test {X_test.shape}')

# =============================================================================
# TREINAR LIGHTGBM
# =============================================================================
clf = lgb.LGBMClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.05,
    class_weight='balanced',
    random_state=SEED,
    verbose=-1
)

clf.fit(X_train, y_train)
print('[5/5] LightGBM treinado!')

# =============================================================================
# SUBMISSÃO
# =============================================================================
predictions = clf.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('✅ CONCLUÍDO: submission.csv')
print(submission['target'].value_counts().sort_index())